In [5]:
import os
import numpy as np
from scipy.stats import pearsonr
from sklearn.model_selection import GroupKFold, cross_val_predict
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline


# CsPIP training

In [ ]:
# loading data
datadir = '0data'
train_y = np.load(os.path.join(datadir,'d1_train_y.npy'))
test_y = np.load(os.path.join(datadir,'d1_test_y.npy'))
train_pred = np.load(os.path.join(datadir,'d1_train_pred.npy'))
test_pred = np.load(os.path.join(datadir,'d1_test_pred.npy'))
train_set = np.load(os.path.join(datadir,'d1_train_set.npy'))
test_set = np.load(os.path.join(datadir,'d1_test_set.npy'))
train_groups = np.load(os.path.join(datadir,'d1_train_group.npy'))
qh_lp_copes = np.load(os.path.join(datadir,'d2_qh_lp_set.npy'))
qh_hp_copes = np.load(os.path.join(datadir,'d2_qh_hp_set.npy'))
qh_lp_pred = np.load(os.path.join(datadir,'d2_qh_lp_pred.npy'))
qh_hp_pred = np.load(os.path.join(datadir,'d2_qh_hp_pred.npy'))
qh_lp_y = np.load(os.path.join(datadir,'d2_qh_lp_y.npy'))
qh_hp_y = np.load(os.path.join(datadir,'d2_qh_hp_y.npy'))


In [3]:
lassopcr = Pipeline(steps=[
                           ('scaler', StandardScaler()),
                           ('pca', PCA()),
                           ('lasso', Lasso())])

# Set up the cross_valdation

cv = GroupKFold(n_splits=10)

# 并行得到逐样本的交叉验证预测
y_pred = cross_val_predict(
    lassopcr,            # 你的 Pipeline
    X=train_set,
    y=train_y,
    groups=train_groups,
    cv=cv,
    n_jobs=-1,           # -1 = 用完所有 CPU 核
    method="predict"     # 调用 predict
)

# 计算总体指标
r      = pearsonr(train_y, y_pred)[0]
rmse   = np.sqrt(mean_squared_error(train_y, y_pred))
r2     = r2_score(train_y, y_pred)
print(r, rmse, r2)
lassopcr.fit(train_set, train_y)
print(pearsonr(test_y, lassopcr.predict(test_set)))
print('-'*20)
print(pearsonr(qh_lp_y,lassopcr.predict(qh_lp_copes)))
print(pearsonr(qh_hp_y,lassopcr.predict(qh_hp_copes)))

0.35521490257490257 1.004722370038097 0.1120164277933402
PearsonRResult(statistic=np.float64(0.4670142128674764), pvalue=np.float64(0.00012999202320807405))
--------------------
PearsonRResult(statistic=np.float64(0.3382695270965094), pvalue=np.float64(0.03517839795001519))
PearsonRResult(statistic=np.float64(0.5168412628472525), pvalue=np.float64(0.0007550686759007282))


# bootstrap

In [ ]:
nbootstraps = 10000
nstop = 200 # frequency to stop/save bootstraps
njobs = -1
brain_v = len_brain
spial_v = len_spinal

from joblib import Parallel, delayed
from scipy.stats import norm
from tqdm import tqdm
from os.path import join as opj
outpath = '/mnt/lxm2/2025/03_Tens/lassopcr_res_0503_2a/boots_new'
name = 'lassopcr'
os.makedirs(opj(outpath, 'permsamples'), exist_ok=True)
os.makedirs(opj(outpath, 'bootsamples'), exist_ok=True)
def bootstrap_weights(X, Y,rs):
    # Randomly select observations
    rng = np.random.RandomState(rs)

    boot_ids = rng.choice(np.arange(len(X)),
                                size=len(X),
                                replace=True)
    try:
        # Fit the classifier on this sample and get the weights
        lassopcr.fit(X[boot_ids], Y[boot_ids])

        # Return the weights and stats
        return np.dot(lassopcr['pca'].components_.T, lassopcr['lasso'].coef_)
    except:
        print("SVD failed on a bootstrap sample. Skipping...")
        return rs  # 返回 None 而不是零向量
# Run in parrallel and stop/save regurarly to run in multiple
for i in tqdm(range(nbootstraps//nstop)):
    # Check if file alrady exist in case bootstrap done x times
    outbootfile = ['bootsamples_' + str(nstop) + 'samples_'
                    + str(i+1) + '_of_'
                    + str(nbootstraps//nstop) + '.npy']
    print("Running permloop " + str(i + 1) + ' out of ' + str(nbootstraps//nstop))
    if not os.path.exists(opj(outpath, 'bootsamples', outbootfile[0])):
        bootstrapped = Parallel(n_jobs=40,
                                verbose=0)(delayed(bootstrap_weights)(X=train_set, Y=train_y,rs=i*200+ii)
                                           for ii in range(nstop))
        try:
            bootstrapped = np.stack(bootstrapped)
        except:
            for ind, vv in enumerate(bootstrapped):
                if isinstance(vv, int):  # Check if vv is of type int
                    # Replace vv with the result of bootstrap_weights
                    bootstrapped[ind] = bootstrap_weights(all_copes, all_y, vv)
        bootstrapped = np.stack(bootstrapped)
        np.save(opj(outpath, 'bootsamples', outbootfile[0]), bootstrapped)


In [ ]:
mask_img = nb.load('/mnt/lxm2/2025/03_Tens/3group_level/all_lr+.gfeat/all_lr_mask_thr31_bin.nii.gz')
spinal_mask = nb.load('/home/lxm/2_lxm/2025/03_Tens/LASSOPCR/group_mask_gm.nii.gz')

# Load all boostraps
bootstrapped = np.vstack(np.stack([np.load(opj(outpath, 'bootsamples', f))
                         for f in os.listdir(opj(outpath, 'bootsamples'))
                         if 'bootsamples' in f], axis=0))

assert bootstrapped.shape[0] == nbootstraps

# Get bootstraped statistics and threshold (as in nltools)
# Zscore
boot_z = bootstrapped.mean(axis=0)/bootstrapped.std(axis=0)

# boot_z[bootstrapped.mean(axis=0) == 0] = 0
unmask(boot_z[:brain_v], mask_img).to_filename(opj(outpath, 'brain_'+name + '_bootz.nii'))
unmask(boot_z[brain_v:], spinal_mask).to_filename(opj(outpath, 'spinal_'+name + '_bootz.nii'))

# P vals
boot_pval =  2 * (1 - norm.cdf(np.abs(boot_z)))
unmask(boot_pval[:brain_v], mask_img).to_filename(opj(outpath, 'brain_'+name + '_boot_pvals.nii'))
unmask(boot_pval[brain_v:], spinal_mask).to_filename(opj(outpath, 'spinal_'+name + '_boot_pvals.nii'))

def fdr(p, q=0.05):
    s = np.sort(p)
    nvox = p.shape[0]
    null = np.array(range(1, nvox + 1), dtype="float") * q / nvox
    below = np.where(s <= null)[0]
    return s[max(below)] if len(below) else -1 # p_fdr
# FDR orrected z
boot_z_fdr = np.where(boot_pval < fdr(boot_pval, q=0.05), boot_z, 0)
boot_z_unc001 = np.where(boot_pval < 0.001, boot_z, 0)
boot_z_unc005 = np.where(boot_pval < 0.005, boot_z, 0)
boot_z_unc01 = np.where(boot_pval < 0.01, boot_z, 0)


unmask(boot_z_fdr[:brain_v], mask_img).to_filename(opj(outpath,
                                              'brain'+ name + '_bootz_fdr05.nii'))
unmask(boot_z_unc001[:brain_v], mask_img).to_filename(opj(outpath,
                                                  'brain'+ name + '_bootz_unc001.nii'))
unmask(boot_z_unc005[:brain_v], mask_img).to_filename(opj(outpath,
                                                    'brain'+ name + '_bootz_unc005.nii'))
unmask(boot_z_unc01[:brain_v], mask_img).to_filename(opj(outpath,
                                                    'brain'+ name + '_bootz_unc01.nii'))
unmask(boot_z_fdr[brain_v:], spinal_mask).to_filename(opj(outpath,
                                               'spianl'+ name + '_bootz_fdr05.nii'))
unmask(boot_z_unc001[brain_v:], spinal_mask).to_filename(opj(outpath,
                                                  'spianl'+ name + '_bootz_unc001.nii'))
unmask(boot_z_unc005[brain_v:], spinal_mask).to_filename(opj(outpath,
                                                    'spianl'+ name + '_bootz_unc005.nii'))
unmask(boot_z_unc01[brain_v:], spinal_mask).to_filename(opj(outpath,
                                                    'spianl'+ name + '_bootz_unc01.nii'))

In [ ]:
boot_z_fdr_brain = np.where(boot_pval[:brain_v] < fdr(boot_pval[:brain_v], q=0.05), boot_z[:brain_v], 0)
unmask(boot_z_fdr_brain, mask_img).to_filename(opj(outpath, 'only_brain_'+name + '_bootz_fdr05.nii'))
boot_z_fdr_spinal = np.where(boot_pval[brain_v:] < fdr(boot_pval[brain_v:], q=0.05), boot_z[brain_v:], 0)
unmask(boot_z_fdr_spinal, spinal_mask).to_filename(opj(outpath, 'only_spinal_'+name + '_bootz_fdr05.nii'))

# Orignial Pipeline

In [ ]:

# all_sub = open('/home/lxm/2_lxm/2025/03_Tens/scripts/all.list').read().splitlines()
# r_list = [x for x in all_sub if '_r' in x]
# l_list = [x for x in all_sub if '_l' in x]
# y_all_r = np.array([np.loadtxt(f'/mnt/lxm2/2025/03_Tens/0data/time_pr/{sub}_pre_pr.txt').mean() for sub in r_list])
# y_all_l = np.array([np.loadtxt(f'/mnt/lxm2/2025/03_Tens/0data/time_pr/{sub}_pre_pr.txt').mean() for sub in l_list])
# import numpy as np
# import nibabel as nb
# from nilearn.image import binarize_img,threshold_img
# from joblib import Parallel, delayed

# brain_mask_img  = nb.load('/mnt/lxm2/2025/03_Tens/3group_level/all_lr+.gfeat/all_lr_mask_thr31_bin.nii.gz')
# spinal_mask_img = nb.load('/home/lxm/2_lxm/2025/03_Tens/LASSOPCR/group_mask_gm.nii.gz')
# brain_mask  = brain_mask_img.get_fdata().astype(bool).ravel()   # (n_vox_brain,)
# spinal_mask = spinal_mask_img.get_fdata().astype(bool).ravel()  # (n_vox_spinal,)

# # 把两张掩膜长度记录下来，后面拼接用
# len_brain, len_spinal = brain_mask.sum(), spinal_mask.sum()

# def load_subject(sub_id, prefix):
#     """
#     读一个受试者的脑 + 脊髓 COPE，返回 (len_brain + len_spinal,) 的 1-D 特征向量
#     """
#     # 路径
#     brain_p  = f'{prefix}/brain/{sub_id}.feat/reg_standard/stats/cope1.nii.gz'
#     spinal_p = f'{prefix}/spinal/{sub_id}.feat/stats/cope1_template.nii.gz'
    
#     brain_data  = nb.load(brain_p).get_fdata().ravel()[brain_mask]     # (len_brain,)
#     spinal_data = nb.load(spinal_p).get_fdata().ravel()[spinal_mask]   # (len_spinal,)
    
#     return np.concatenate((brain_data, spinal_data), axis=0)

# # ── 2. 并行读取 ─────────────────────────────────────────────
# prefix = '/mnt/lxm2/2025/03_Tens/2fst_level'

# # 左右两组
# all_data = Parallel(n_jobs=-1, backend='loky', verbose=5)(
#     delayed(load_subject)(sub, prefix) for sub in l_list
# )
# test_data = Parallel(n_jobs=-1, backend='loky', verbose=5)(
#     delayed(load_subject)(sub, prefix) for sub in r_list
# )

# all_data  = np.vstack(all_data)   # shape = (n_L, len_brain + len_spinal)
# test_data = np.vstack(test_data)  # shape = (n_R, len_brain + len_spinal)

# # ── 3. 组装后续矩阵 / 标签 / 分组 ───────────────────────────
# # all_copes = np.vstack((all_data, test_data))
# all_copes = np.vstack((all_data, test_data))
# all_y     = np.hstack((y_all_l, y_all_r))
# groups    = np.tile(np.arange(len(y_all_l)), 2)
# from sklearn.model_selection import GroupShuffleSplit
# group_split = GroupShuffleSplit(n_splits=2, test_size=0.33, random_state=42)

# # 拆分数据集，确保同一组数据不会同时出现在训练集和测试集中
# for train_index, test_index in group_split.split(all_copes, groups=groups):
#     train_set = all_copes[train_index]
#     train_y = all_y[train_index]
#     test_set = all_copes[test_index]
#     test_y = all_y[test_index]
#     train_groups = groups[train_index]
#     test_groups = groups[test_index]
#     train_id = [r_list[x] for x in train_groups]
#     test_id = [r_list[x] for x in test_groups]
# outdir = '/home/lxm/2_lxm/2025/03_Tens/manuscript_FINAL/0data'
# if not os.path.exists(outdir):
#     os.makedirs(outdir)
# np.save(os.path.join(outdir,'d1_train_set.npy'),train_set)
# np.save(os.path.join(outdir,'d1_test_set.npy'),test_set)
# np.save(os.path.join(outdir,'d1_train_y.npy'),train_y)
# np.save(os.path.join(outdir,'d1_train_pred.npy'),y_pred)
# np.save(os.path.join(outdir,'d1_test_y.npy'),test_y)
# np.save(os.path.join(outdir,'d1_test_pred.npy'),lassopcr.predict(test_set))
# np.save(os.path.join(outdir,'d2_qh_lp_pred.npy'),lassopcr.predict(qh_lp_copes))
# np.save(os.path.join(outdir,'d2_qh_hp_pred.npy'),lassopcr.predict(qh_hp_copes))
# np.save(os.path.join(outdir,'d2_qh_hp_y.npy'),real_hp)
# np.save(os.path.join(outdir,'d2_qh_lp_y.npy'),real_lp)
# np.save(os.path.join(outdir,'d1_train_group.npy'),train_groups)
# np.save(os.path.join(outdir,'d1_test_group.npy'),test_groups)
# np.save(os.path.join(outdir,'d2_qh_lp_pred.npy'), qh_lp_copes)
# np.save(os.path.join(outdir,'d2_qh_hp_pred.npy'), qh_hp_copes)
